In [236]:
import random
import numpy as np

from datetime import timedelta
import math
from utilities import generate_random_date, generate_licence_plate

from places import RandomPlaceGenerator

In [237]:
car_brands = {
  "Bmw": ["Series 3"],
  "Toyota": ["Corolla", "Land Cruiser", "Hilux", "Rav"],
  "Skoda": ["Superb"],
  "Kia": ["Stinger GT", "Ceed 3"],
  "Volkswagen": ["Crafter", "Passat"],
  "Hyundai": ["Ionia", "Tucson"]
}

motorbike_brands = {
  "Triumph": ["Tiger 1050"],
  "Kawasaki": ["Versys 1000"],
  "BMW": ["R 1250 RT"],
}

In [238]:
from numpy import loadtxt
names_male = loadtxt("./data/supporting/names/polish_male_firstnames.txt", dtype='str')
names_female = loadtxt("./data/supporting/names/polish_female_firstnames.txt", dtype='str')
surnames = loadtxt("./data/supporting/names/polish_surnames.txt", dtype='str')
genders = ["female", "male"]
ranks = [
  "Constable",
  "Senior constable",
  "Sergeant",
  "Senior sergeant",
  "Staff sergeant",
  "Junior aspirant",
  "Aspirant",
  "Senior aspirant",
  "Staff aspirant",
  "Deputy commissioner",
  "Commissioner",
  "Chief commissioner",
  "Deputy inspector",
  "Junior inspector",
  "Inspector",
  "Chief inspector",
  "Inspector general"
]

print(len(names_male))
print(len(names_female))
print(len(surnames))

300
300
314549


# Define object classes

In [239]:
class Position:
  def __init__(self, lat, lng):
    
    self.lat = lat
    self.lng = lng

In [240]:
class Officer:
  def __init__(self, officer_id, first_name, last_name, rank, gender, date_of_birth, hire_date, email, phone_number, city):
    
    self.officer_id = officer_id
    self.first_name = first_name
    self.last_name = last_name
    self.rank = rank
    self.gender = gender
    self.date_of_birth = date_of_birth
    self.hire_date = hire_date
    self.email = email
    self.phone_number = phone_number
    
    self.city = city
    
    self.team = None

In [241]:
# In seconds +- 50%
# 25 minutes
INCIDENT_RESOLUTION_TIME = 1500

class Vehicle:
  def __init__(self, license_plate_number, vehicle_type, brand, model, manufacturing_year, position, city):
    
    # self.vehicle_id = vehicle_id
    self.license_plate_number = license_plate_number
    
    # "car" or "motorbike"
    self.vehicle_type = vehicle_type
    self.brand = brand
    self.model = model
    self.manufacturing_year = manufacturing_year
    self.position = position
    
    # meters per second
    self.last_inspection = manufacturing_year
    
    self.assigned_incident = None
    self.is_resolving_incident = False
    self.time_till_resolved = 0
    
    self.team = None
    self.city = city
    
    # In seconds, 0 = to be freed
    self.team_time = 0

In [242]:
class Team:
  def __init__(self, vehicle, start_datetime, officers):
    
    self.vehicle = vehicle
    self.start_datetime = start_datetime
    self.end_datetime = None
    
    self.officers = officers

In [243]:
class Incident:
  def __init__(self, incident_id, city, district, report_datetime, description, position):
    
    self.incident_id = incident_id
    self.city = city
    self.district = district
    self.report_datetime = report_datetime
    self.arrival_datetime = None
    self.victim_satisfaction = None
    self.description = description
    self.incident_type = None
    self.position = position
    
    self.victims = list()

In [244]:
class Victim:
  def __init__(self, victim_id, first_name, last_name, gender, date_of_birth):
    
    self.victim_id = victim_id
    self.first_name = first_name
    self.last_name = last_name
    self.gender = gender
    self.date_of_birth = date_of_birth

In [245]:
class VehiclePosition:
  def __init__(self, vehicle, position, time):
    
    self.vehicle = vehicle
    self.position = position
    self.time = time

In [246]:
class City:
  def __init__(self, city_name, top_left, bottom_right):
    
    self.city_name = city_name
    self.top_left = top_left
    self.bottom_right = bottom_right

# Define Generator

In [247]:
def calculate_distance(pos_1, pos_2):
  return math.sqrt((pos_1.lat - pos_2.lat)**2 + (pos_1.lng - pos_2.lng)**2)

In [248]:
# Both ends included
def random_range(start, end, INTERNAL_SUBSTEPS=1000):
  INTERNAL_SUBSTEPS = 1000
  return random.randrange(int(start*INTERNAL_SUBSTEPS), int(end*INTERNAL_SUBSTEPS)+1, 1) / INTERNAL_SUBSTEPS

In [249]:
def generate_personal_info():
  gender = random.choice(genders)
  surname = random.choice(surnames)
  first_name = ""
  
  if gender == "male":
    first_name = random.choice(names_male)
  else:
    first_name = random.choice(names_female)
    
  return {
    "gender": gender,
    "first_name": first_name,
    "surname": surname
  }  

In [250]:
# In order of cities
INITIAL_COUNTS = {
  "officers": [60, 80, 50],
  "cars": [35, 50, 45],
  "motorbikes": [10, 15, 5]
}

MERCATOR_PER_METER = 2.245789145352464e-6

# 30 meters
RANDOM_POS_STEP = MERCATOR_PER_METER * 30


class Generator:
  
  def __init__(self, initial_counts, initial_date, simulation_timestep, incidents_per_hour):
    
    self.initial_counts = initial_counts
    
    # In seconds
    self.simulation_timestep = simulation_timestep
    self.incidents_per_hour = incidents_per_hour
    
    
    self.vehicles = list()
    self.officers = list()
    
    self.incidents = list()
    self.victims = list()
    
    self.teams = list()
    self.vehicle_positions = list()
    
    self.current_time = initial_date
    
    self.cities = list()
    
    # city, top_left_latitude, top_left_longitude, bottom_right_latitude,  bottom_right_longitude
    # Gdańsk: 54.44725173693497,  18.44234779682661,  54.2749,  18.9362
    # Warszawa: 52.3657,  20.8515,  52.1033,  21.2690
    # Kraków: 50.1233, 19.8094,  49.9737,  20.2150

    self.cities.append(City("Gdansk", Position(54.44725173693497, 18.44234779682661), Position(54.2749, 18.9362)))
    self.cities.append(City("Warszawa", Position(52.3657, 20.8515), Position(52.1033, 21.2690)))
    self.cities.append(City("Krakow", Position(50.1233, 19.8094), Position(49.9737, 20.2150)))
  
  def create_officer(self, id, city):
    MIN_OFFICER_AGE = 21
    MAX_OFFICER_AGE = 70
    MAX_OFFICER_HIRE_TIME = 35
    
    personal_info = generate_personal_info()

    hire_date = generate_random_date(self.current_time - timedelta(days=MAX_OFFICER_HIRE_TIME*365), self.current_time)
    
    date_of_birth = generate_random_date(self.current_time - timedelta(days=MAX_OFFICER_AGE*365), hire_date - timedelta(days=MIN_OFFICER_AGE*365))
    
    rank = random.choice(ranks)
    
    officer = Officer(id, personal_info["first_name"], personal_info["surname"], rank, personal_info["gender"], date_of_birth, hire_date, "officer{}@police.org.pl".format(id), 111111111, city)
    
    return officer
  
  
  def create_car(self, city):
    
    OLDEST_CAR_YEAR = 10
    EARLIEST_CAR_YEAR = 1
    
    license_plate_number = generate_licence_plate()
    brand = random.choice(list(car_brands.keys()))
    model = random.choice(car_brands[brand])
    manufacturing_year = generate_random_date(self.current_time - timedelta(days=OLDEST_CAR_YEAR*365), self.current_time - timedelta(days=EARLIEST_CAR_YEAR*365))
    
    # FIX it - just an testing position
    lng = random_range(city.top_left.lng, city.bottom_right.lng, 10000)
    lat = random_range(city.bottom_right.lat, city.top_left.lat, 10000)
    position = Position(lat, lng)
    
    car = Vehicle(license_plate_number, "car", brand, model, manufacturing_year, position, city)
    
    return car
  
  def create_motorbike(self, city):
    OLDEST_CAR_YEAR = 10
    EARLIEST_CAR_YEAR = 1
    
    license_plate_number = generate_licence_plate()
    brand = random.choice(list(motorbike_brands.keys()))
    model = random.choice(motorbike_brands[brand])
    manufacturing_year = generate_random_date(self.current_time - timedelta(days=OLDEST_CAR_YEAR*365), self.current_time - timedelta(days=EARLIEST_CAR_YEAR*365))
    
    # FIX it - just an testing position
    lng = random_range(city.top_left.lng, city.bottom_right.lng, 10000)
    lat = random_range(city.bottom_right.lat, city.top_left.lat, 10000)
    position = Position(lat, lng)
    
    motorbike = Vehicle(license_plate_number, "motorbike", brand, model, manufacturing_year, position, city)
    
    return motorbike
  
  def generate_initial_data(self):
    
    for c_i, city in enumerate(self.cities):
      
      for i in range(0, self.initial_counts["officers"][c_i]):
        officer = self.create_officer(i, city)
        self.officers.append(officer)
        
      for i in range(0, self.initial_counts["cars"][c_i]):
        car = self.create_car(city)
        self.vehicles.append(car)
        
      for i in range(0, self.initial_counts["motorbikes"][c_i]):
        motorbike = self.create_motorbike(city)
        self.vehicles.append(motorbike)
  
  
  def move_vehicle(self, vehicle):
    
    if vehicle.team == None:
      return
    
    # In m/s
    DEFAULT_VEHICLE_SPEED = 14
    # EMERGENCY_CAR_SPEED = 25
    # EMERGENCY_MOTORBIKE_SPEED = 30

    DELTA_MERCATOR_MAIN = self.simulation_timestep * DEFAULT_VEHICLE_SPEED * MERCATOR_PER_METER

    if vehicle.assigned_incident == None and vehicle.is_resolving_incident == False:
      # Going randomly, not occupied
      delta_lng = 0
      delta_lat = 0
      
      axis = random.choice([True, False])
      direction = random.choice([-1, 1])
      
      distance = random.randrange(int(DELTA_MERCATOR_MAIN / RANDOM_POS_STEP * 0.75), int(DELTA_MERCATOR_MAIN / RANDOM_POS_STEP * 1.25)+1, 1) * RANDOM_POS_STEP
      
      # print(DELTA_MERCATOR_MAIN)
      if axis:
        delta_lng = distance * direction
      else:
        delta_lat = distance * direction
        
      # Check city boudaries
      
      if (vehicle.position.lng + delta_lng < vehicle.city.top_left.lng or
        vehicle.position.lng + delta_lng > vehicle.city.bottom_right.lng or
        vehicle.position.lat + delta_lat < vehicle.city.bottom_right.lat or
        vehicle.position.lat + delta_lat > vehicle.city.top_left.lat):
        # We inverse the direction
        direction = -1 * direction
      
      vehicle.position.lng += delta_lng
      vehicle.position.lat += delta_lat
    elif vehicle.assigned_incident != None and vehicle.is_resolving_incident == False:
      # Goes to the incident
      
      d_lat = vehicle.assigned_incident.position.lat - vehicle.position.lat
      d_lng = vehicle.assigned_incident.position.lng - vehicle.position.lng
      dist_to_incident = math.sqrt((d_lat)**2 + (d_lng)**2)
      
      step_distance = random.randrange(int(DELTA_MERCATOR_MAIN / RANDOM_POS_STEP * 0.75), int(DELTA_MERCATOR_MAIN / RANDOM_POS_STEP * 1.25)+1, 1) * RANDOM_POS_STEP
      
      if step_distance > dist_to_incident:        
        vehicle.position.lat = vehicle.assigned_incident.position.lat
        vehicle.position.lng = vehicle.assigned_incident.position.lng
        
        # Calculate arrival time depending on travelled distance to distance to incident proportions
        distances_ratio = dist_to_incident / step_distance
        
        vehicle.assigned_incident.arrival_datetime = self.current_time + timedelta(seconds=self.simulation_timestep * distances_ratio)
        
        vehicle.is_resolving_incident = True
        vehicle.time_till_resolved = random_range(INCIDENT_RESOLUTION_TIME * 0.5, INCIDENT_RESOLUTION_TIME * 1.5)
        
        # print("Stop and handle incident, time left: ", vehicle.time_till_resolved)
      else:
        
        dir_lat = d_lat / dist_to_incident
        dir_lng = d_lng / dist_to_incident
      
        vehicle.position.lat += dir_lat * step_distance
        vehicle.position.lng += dir_lng * step_distance
        
        # print("Moved, distance to incident = ", dist_to_incident)  
    elif vehicle.assigned_incident != None and vehicle.is_resolving_incident == True:
      # Is resolving incident
      vehicle.time_till_resolved -= self.simulation_timestep
      
      # print("counter decreases, time left = {}".format(vehicle.time_till_resolved))
      
      if vehicle.time_till_resolved <= 0:
        
        # print("Incident nr {} resolved".format(vehicle.assigned_incident.incident_id))
        
        vehicle.time_till_resolved = 0
        vehicle.is_resolving_incident = False
        vehicle.assigned_incident = None
        
        # TODO Move proportionally to the time left
        # TODO Update incident with satisfaction score
           
  def move_vehicles(self):
    
    for vehicle in self.vehicles:
      self.move_vehicle(vehicle)
    
  def generate_victims_for_incident(self, incident):
    victims_count = np.random.poisson(lam=0.4, size=1)[0]+1
    
    date_of_birth = generate_random_date(self.current_time - timedelta(days=90*365), self.current_time - timedelta(days=2*365))
    
    for i in range(victims_count):
      
      personal_info = generate_personal_info()
    
      victim = Victim(len(self.victims), personal_info["first_name"], personal_info["surname"], personal_info["gender"], date_of_birth)
      self.victims.append(victim)
      incident.victims.append(victim)
  
  def save_vehicle_positions(self):
    for vehicle in self.vehicles:
      pos_copy = Position(vehicle.position.lat, vehicle.position.lng)
      self.vehicle_positions.append(VehiclePosition(vehicle, pos_copy, self.current_time))
      
  def count_vehicles(self):
    assigned = 0
    resolving = 0
    
    for vehicle in self.vehicles:
      
      if vehicle.assigned_incident != None:
        assigned += 1
      
      if vehicle.is_resolving_incident == True:
        resolving += 1
  
  
  def get_city_officers(self, city):
    result = list()
    for officer in self.officers:
      if officer.city == city:
        result.append(officer)
        
    return result
  
  def get_city_vehicles(self, city):
    result = list()
    for vehicle in self.vehicles:
      if vehicle.city == city:
        result.append(vehicle)
        
    return result
  
  def update_teams(self):
    for vehicle in self.vehicles:
      
      if vehicle.team != None:
        if vehicle.team_time <= 0:
          # FIX END DATE could be varying within timestep
          vehicle.team.end_datetime = self.current_time
          # print("team disapears at ", vehicle.team.end_datetime)
          
          for officer in vehicle.team.officers:
            officer.team = None
          
          vehicle.team = None
          # TODO Could add some cooldown for the vehicle
        else:
          vehicle.team_time -= self.simulation_timestep
  

  def select_officers_to_team(self, vehicle, officers):
    
    team = list()
    available = list()
    
    officer_count = 1
    if vehicle.vehicle_type != "motorbike":
      officer_count = random.choice([1, 2, 3])
      
    for officer in officers:
      if officer.team == None:
        available.append(officer)
      
    if officer_count <= len(available):
      for i in range(officer_count):
        # print(len(available))
        team.append(available[i])
    else:
      for officer in available:
        team.append(officer)
        
    return team
 
  def assign_team(self, vehicle, city):
    # In seconds => 8 hours = 28800s
    AVARAGE_TEAM_TIME = 28800
    
    if vehicle.team == None:
      officers = self.get_city_officers(city)
      
      chosen_officers = self.select_officers_to_team(vehicle, officers)
      
      if len(chosen_officers) > 0:
        team = Team(vehicle, self.current_time, chosen_officers)
        vehicle.team = team
        vehicle.team_time = random_range(AVARAGE_TEAM_TIME * 0.75, AVARAGE_TEAM_TIME * 1.25, 4000)
        self.teams.append(team)
        
        for officer in chosen_officers:
          officer.team = team
      # else:
        # print("No more available officers")
    # else:
    #   print("vehicle is full")
      
  def assign_teams(self):
    
    for city in self.cities:
      # Assign officers to vehicles within the city
      
      # officers = self.get_city_officers(city)
      vehicles = self.get_city_vehicles(city)
      
      # print("Assign officers to vehicles within the city ", len(officers), len(vehicles))
      
      for vehicle in vehicles:
        self.assign_team(vehicle, city)
      
        
  def assign_vehicle_to_incident(self, incident):   
    closest_vehicle = None #self.vehicles[0]
    closest_dist = 0 # calculate_distance(incident.position, closest_vehicle.position)
   
    for vehicle in self.vehicles:
        
      if vehicle.assigned_incident == None and vehicle.is_resolving_incident == False:
        distance = calculate_distance(incident.position, vehicle.position)
        
        if closest_vehicle == None or distance < closest_dist:
          closest_dist = distance
          closest_vehicle = vehicle
        
    # We know the closest vehicle to the incident, now we assign it
    
    if closest_vehicle != None:
      closest_vehicle.assigned_incident = incident
    # else:
      # print("All vehicles are currently occupied")
  
  def generate_incidents(self):
    avarage_incidents = self.simulation_timestep / 3600 * self.incidents_per_hour
    # Here, higher random range of 50%
    incidents_count = int(random_range(int(avarage_incidents * 0.5), int(avarage_incidents * 1.5), 1))
    # print(incidents_count)
    
    for i in range(incidents_count):      
      
      place_gen = RandomPlaceGenerator()
      city_name, district_name, selected_point = place_gen.select_random_place()
      position = Position(selected_point[0], selected_point[1])
      
      incident_datetime = self.current_time + timedelta(seconds=self.simulation_timestep * random_range(0, 1, 1000))
      # TODO Should add victims to incident
      
      incident = Incident(len(self.incidents), city_name, district_name, incident_datetime, None, position)
      self.generate_victims_for_incident(incident)
      
      # TODO Should assign car team to incident and add victims
      self.assign_vehicle_to_incident(incident)
      
      self.incidents.append(incident)
    
  def simulate(self, start_datetime, end_datetime):
    time_diff = end_datetime - start_datetime
    iterations = time_diff.total_seconds() / self.simulation_timestep
    
    timestep = timedelta(seconds=self.simulation_timestep)

    for i in range(0, int(iterations)):
      self.save_vehicle_positions()
      
      self.generate_incidents()
      self.move_vehicles()
      
      self.update_teams()
      self.assign_teams()
      
      self.current_time = start_datetime + i * timestep

In [251]:
from datetime import datetime
startdate = datetime(2023, 1, 1)
enddate = datetime(2023, 1, 31)

generator = Generator(INITIAL_COUNTS, startdate, 600, 16)
generator.generate_initial_data()

# generator.assign_teams()
# 
generator.simulate(startdate, enddate)

In [252]:
print(len(generator.incidents), len(generator.vehicle_positions))
print(len(generator.officers), len(generator.vehicles), len(generator.teams))

# print(generator.vehicles[0].position.lat, generator.vehicles[0].position.lng)
# for team in generator.teams:
#   print(team.start_datetime, team.end_datetime)

# print(generator.vehicle_positions[130114].time)
# for veh_pos in generator.vehicle_positions:
#   print(veh_pos.position.lat, veh_pos.position.lng, veh_pos.vehicle)
  
# for officer in generator.officers:
#   print(officer.team)
  
# for vehicle in generator.vehicles:
#   print(vehicle.team)
  # print(vehicle.team.start_datetime, vehicle.team.end_datetime)

8774 691200
190 160 11340
